In [1]:
from torchdata.datapipes.iter import FileLister, FileOpener
import os

# Set the current working directory
os.chdir("/data/workspace/Codes/CAFE")


/data/workspace/app/miniconda3/envs/cafe/lib/python3.8/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


关于Example的格式
* 每个example是一个用户的29条广告记录。
* 用户特征可能是单值特征，也可能是多值特征。广告的特征都是单值特征。
* part_1前缀代表用户侧特征，part_2前缀代表广告侧特征。
* keys后缀是需要的特征id，values后缀内容在这里都是1，可以忽略。

用户侧，每个field中包括的是当前用户的特征；广告侧，每个field包含的是29条广告的特征，故广告侧每个field查出来是一个29维的tensor

In [2]:
#  指定TFRecord文件路径
tfrecord_file = "*.tfr"

# 创建FileLister和FileOpener DataPipe
datapipe1 = FileLister("datasets/moments", tfrecord_file)
datapipe2 = FileOpener(datapipe1, mode="b")

# 使用TFRecordLoader加载TFRecord文件
tfrecord_loader_dp = datapipe2.load_from_tfrecord()

# 遍历并打印TFRecord中的示例
for example in tfrecord_loader_dp:
    print(example)

{'part_1_field_7525/values': tensor([1.]), 'part_1_field_11472/values': tensor([1.]), 'part_1_field_5807/keys': tensor([413850034111581323, 789247087774248954, 519427337367869418,
        325382001630508417,  32583946403909251,  16511267877847046,
        882325335566955700, 318214712678210516, 376562865917637877,
          7437225237854024]), 'part_1_field_6166/keys': tensor([412152340294530059]), 'part_1_field_11878/values': tensor([1.]), 'part_1_field_3658/keys': tensor([292669397743979535]), 'part_1_field_4299/keys': tensor([462540201980748742, 256948630425831839]), 'part_1_field_5827/values': tensor([1.]), 'part_1_field_4225/values': tensor([1.]), 'part_1_field_5825/keys': tensor([505634319209378819]), 'part_1_field_17927/keys': tensor([491385718456518896, 195175916182320944]), 'part_1_field_11470/keys': tensor([93395046810123363]), 'part_1_field_3024/values': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'part_1_field_3572/keys': tensor([ 504489548241999740, 1106166001228940250, 

模型用到的特征放置在`datasets/moments/gpu_moment_gpu_v506_fix_v3/conf/global_conf.json`文件下，从这里读用到了哪些特征

In [6]:
import json

# Define the path to the JSON file
json_file_path = "datasets/moments/gpu_moment_gpu_v506_fix_v3/conf/global_conf.json"

# Read the JSON file
with open(json_file_path, 'r') as json_file:
    features = json.load(json_file)

# Check the keys in the features dictionary
feature_keys = features.keys()
print("Keys in features:", feature_keys)
input_columns = (features['input_columns'])
print("input_columns:", input_columns)


Keys in features: dict_keys(['embed', 'model', 'input_columns', 'aux_fields'])
input_columns: {'ad_sparse': ['product_type_IntIndexHash', 'game_type_BytesIndexHash', 'game_core_BytesIndexHash', 'game_theme_BytesIndexHash', 'bid_objective_IntIndexHash', 'second_bid_objective_IntIndexHash', 'advertiser_second_industry_id_IntIndexHash', 'fourth_category_id_wx_IntIndexHash', 'ad_first_category_wx_IntIndexHash', 'fylp_has_buy_index_IntIndexHash', 'fylp_has_text_booking_IntIndexHash', 'f_102_8352_BytesIndexHash', 'f_102_8356_BytesIndexHash', 'second_bid_objective__deep_bid_bucket_BytesIndexHash', 'bid_objective__cpc_bid_bucket_BytesIndexHash', 'bid_objective__cpm_bid_bucket_BytesIndexHash', 'product_type__deep_bid_bucket_BytesIndexHash', 'product_type__cpc_bid_bucket_BytesIndexHash', 'product_type__cpm_bid_bucket_BytesIndexHash', 'ad_second_category_wx_IntIndexHash', 'FMarketingGoal_IntIndexHashWithDefaultV2', 'FMarketingCarrierType_IntIndexHashWithDefaultV2', 'FMarketingTargetType_IntIndexH

In [7]:
field_names = []
for values in input_columns.values():
    field_names.extend(values)
print("field_names:", field_names)
print("number of fields:", len(field_names))

field_names: ['product_type_IntIndexHash', 'game_type_BytesIndexHash', 'game_core_BytesIndexHash', 'game_theme_BytesIndexHash', 'bid_objective_IntIndexHash', 'second_bid_objective_IntIndexHash', 'advertiser_second_industry_id_IntIndexHash', 'fourth_category_id_wx_IntIndexHash', 'ad_first_category_wx_IntIndexHash', 'fylp_has_buy_index_IntIndexHash', 'fylp_has_text_booking_IntIndexHash', 'f_102_8352_BytesIndexHash', 'f_102_8356_BytesIndexHash', 'second_bid_objective__deep_bid_bucket_BytesIndexHash', 'bid_objective__cpc_bid_bucket_BytesIndexHash', 'bid_objective__cpm_bid_bucket_BytesIndexHash', 'product_type__deep_bid_bucket_BytesIndexHash', 'product_type__cpc_bid_bucket_BytesIndexHash', 'product_type__cpm_bid_bucket_BytesIndexHash', 'ad_second_category_wx_IntIndexHash', 'FMarketingGoal_IntIndexHashWithDefaultV2', 'FMarketingCarrierType_IntIndexHashWithDefaultV2', 'FMarketingTargetType_IntIndexHashWithDefaultV2', 'FMarketingTargetIdHash_IntIndexHashWithDefaultV2', 'FMarketingSubTargetIdHa

field的信息在`datasets/moments/algorithm.xml`文件中

In [8]:
import xml.etree.ElementTree as ET

# Define the path to the XML file
xml_file_path = "datasets/moments/algorithm.xml"

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Create a dictionary to store the feature information
feature_info = {}

# Iterate through each feature element in the XML
for feature_element in root.findall('.//feature'):
    alias = feature_element.get('alias')
    
    # Store the information in the dictionary with field as the key
    feature_info[alias] = {
        'field': feature_element.get('field'),
        'id': feature_element.get('id'),
        'is_context': feature_element.get('is_context'),
        'type': feature_element.get('type'),
        'is_multi_value': feature_element.get('is_multi_value'),
        'is_online_feature': feature_element.get('is_online_feature')
    }

# Create a list to store the matched feature information
matched_features = []

# Iterate through each field name and find corresponding information
for field_name in field_names:
    if field_name in feature_info:
        matched_features.append({
            'field_name': field_name,
            'info': feature_info[field_name]
        })
    else:
        print(f"Warning: Field {field_name} not found in the XML file.")

# Print the matched feature information
print("Matched Feature Information:")
for feature in matched_features:
    print(f"Alias: {feature['field_name']}")
    print(f"  Field: {feature['info']['field']}")
    print(f"  ID: {feature['info']['id']}")
    print(f"  Is Context: {feature['info']['is_context']}")
    print(f"  Type: {feature['info']['type']}")
    print(f"  Is Multi Value: {feature['info']['is_multi_value']}")
    print(f"  Is Online Feature: {feature['info']['is_online_feature']}")
    print()

# Print summary
print(f"Total fields: {len(field_names)}")
print(f"Matched fields: {len(matched_features)}")
print(f"Unmatched fields: {len(field_names) - len(matched_features)}")

# Create a set of all field names
all_field_names = set(field_names)

# Create a set of matched field names
matched_field_names = set(feature['field_name'] for feature in matched_features)

# Find unmatched field names
unmatched_field_names = all_field_names - matched_field_names

# Print unmatched field names
print("Unmatched Field Names:")
for field_name in unmatched_field_names:
    print(field_name)

print(f"\nTotal unmatched fields: {len(unmatched_field_names)}")

Matched Feature Information:
Alias: product_type_IntIndexHash
  Field: 3209
  ID: 3209
  Is Context: false
  Type: INT64
  Is Multi Value: true
  Is Online Feature: false

Alias: game_type_BytesIndexHash
  Field: 3285
  ID: 3285
  Is Context: false
  Type: INT64
  Is Multi Value: true
  Is Online Feature: false

Alias: game_core_BytesIndexHash
  Field: 3286
  ID: 3286
  Is Context: false
  Type: INT64
  Is Multi Value: true
  Is Online Feature: false

Alias: game_theme_BytesIndexHash
  Field: 3287
  ID: 3287
  Is Context: false
  Type: INT64
  Is Multi Value: true
  Is Online Feature: false

Alias: bid_objective_IntIndexHash
  Field: 3219
  ID: 3219
  Is Context: false
  Type: INT64
  Is Multi Value: true
  Is Online Feature: false

Alias: second_bid_objective_IntIndexHash
  Field: 3220
  ID: 3220
  Is Context: false
  Type: INT64
  Is Multi Value: true
  Is Online Feature: false

Alias: advertiser_second_industry_id_IntIndexHash
  Field: 3302
  ID: 3302
  Is Context: false
  Type: INT

one matched_feature_instance:
```text
{'field_name': 'product_type_IntIndexHash', 'info': {'field': '3209', 'id': '3209', 'is_context': 'false', 'type': 'INT64', 'is_multi_value': 'true', 'is_online_feature': 'false'}}
```

For each matched feature, We construct the new key_name following rules:
* For user features, i.e., features that `is_context=='true'`, add prefix `part_1_field_`
* For ads features, i.e., features that `is_context=='false'`, add prefix `part_2_field_`
* Add feature id after the prefix
* After the feature id, add postfix `/keys`
* The new key_name for each feature should then be: `${prefix}id${postfix}`

In [10]:
# Construct new key_names for matched features
keys = {'user': [], 'ads': []}

for feature in matched_features:
    is_context = feature['info']['is_context'] == 'true'
    prefix = "part_1_field_" if is_context else "part_2_field_"
    feature_id = feature['info']['id']
    postfix = "/keys"
    
    new_key_name = f"{prefix}{feature_id}{postfix}"
    if is_context:
        keys['user'].append((feature['field_name'], new_key_name, feature['info']))
    else:
        keys['ads'].append((feature['field_name'], new_key_name, feature['info']))

# Print the results
print("New Key Names:")
for part in ['user', 'ads']:
    print(f"{part} part:")
    for original_name, new_name, info in keys[part]:
        print(f"Original: {original_name}")
        print(f"New: {new_name}")
        print(f"Info: {info}")
        print()

tot = 0
for part in ['user', 'ads']:
    tot += len(keys[part])

print(f"Total new key names created: {tot}")


New Key Names:
user part:
Original: wuid_user_feature_model_size_IntIndexHash
New: part_1_field_3580/keys
Info: {'field': '3580', 'id': '3580', 'is_context': 'true', 'type': 'INT64', 'is_multi_value': 'true', 'is_online_feature': 'true'}

Original: wuid_user_feature_original_device_model_IntIndexHash
New: part_1_field_3581/keys
Info: {'field': '3581', 'id': '3581', 'is_context': 'true', 'type': 'INT64', 'is_multi_value': 'true', 'is_online_feature': 'true'}

Original: skip_privacy_types_IntIndexHash
New: part_1_field_3534/keys
Info: {'field': '3534', 'id': '3534', 'is_context': 'true', 'type': 'INT64', 'is_multi_value': 'true', 'is_online_feature': 'true'}

Original: datahubLog_age_IntIndexHash
New: part_1_field_11878/keys
Info: {'field': '11878', 'id': '11878', 'is_context': 'true', 'type': 'INT64', 'is_multi_value': 'true', 'is_online_feature': 'true'}

Original: datahubLog_gender_IntIndexHash
New: part_1_field_11880/keys
Info: {'field': '11880', 'id': '11880', 'is_context': 'true', 

Get the features of each example based on `keys`

Each part of `keys` is a list of tuple, example:
```text
(
    datahubLog_gender_IntIndexHash,
    part_1_field_11880/keys
    {'field': '11880', 'id': '11880', 'is_context': 'true', 'type': 'INT64', 'is_multi_value': 'true', 'is_online_feature': 'true'}
)
```

In [12]:
import copy

# Function to extract features for a single example
def extract_example_features(example, keys_part):
    features = []
    not_found_count = 0  # Counter for features not found
    for original_name, new_name, info in keys_part:
        if new_name in example:
            feature_tensor = example[new_name]
            features.append((original_name, new_name, feature_tensor, info['id']))
        else:
            not_found_count += 1  # Increment counter if feature is not found
    print(f"Total features not found: {not_found_count}")  # Print the count of not found features
    return features

def extract_samples(example, keys):
    samples = []
    user_features = extract_example_features(example, keys['user'])
    ads_features = extract_example_features(example, keys['ads'])
    num_ads_in_each_example = len(ads_features[0][2])
    print(f"num_ads_in_each_example: {num_ads_in_each_example}")
    # print(user_features[0])
    samples = []
    for i in range(num_ads_in_each_example):
        # 第i个广告的特征
        features_i = []
        for feature in ads_features:
            ads_feature_tensor_i = feature[2][i]
            ads_feature_i = (*feature[:2], ads_feature_tensor_i, feature[3])
            features_i.append(ads_feature_i)
        samples.append(user_features + copy.deepcopy(features_i))
    
    return samples

# Extract features for all examples
all_features = []
samples = []
for example in tfrecord_loader_dp:
    samples.extend(extract_samples(example, keys))

print(len(samples))

Total features not found: 25
Total features not found: 6
Total features not found: 27
Total features not found: 6
Total features not found: 25
Total features not found: 6
89


In [14]:
print(samples[0])

[('wuid_user_feature_model_size_IntIndexHash', 'part_1_field_3580/keys', tensor([696766764346525237]), '3580'), ('wuid_user_feature_original_device_model_IntIndexHash', 'part_1_field_3581/keys', tensor([639221652894307112]), '3581'), ('skip_privacy_types_IntIndexHash', 'part_1_field_3534/keys', tensor([112409236250416712]), '3534'), ('datahubLog_age_IntIndexHash', 'part_1_field_11878/keys', tensor([317212975329771157]), '11878'), ('datahubLog_gender_IntIndexHash', 'part_1_field_11880/keys', tensor([845374463820734760]), '11880'), ('mix_user_feature_education_IntIndexHash', 'part_1_field_11893/keys', tensor([788639778177576639]), '11893'), ('mix_user_feature_marriage_status_IntIndexHash', 'part_1_field_11895/keys', tensor([817113568469023338]), '11895'), ('mix_user_feature_working_status_IntIndexHash', 'part_1_field_11897/keys', tensor([598274469633128173]), '11897'), ('mix_user_feature_employment_classification_IntIndexHash', 'part_1_field_11899/keys', tensor([494111864421111778]), '11

In [15]:
print(samples[1])

[('wuid_user_feature_model_size_IntIndexHash', 'part_1_field_3580/keys', tensor([696766764346525237]), '3580'), ('wuid_user_feature_original_device_model_IntIndexHash', 'part_1_field_3581/keys', tensor([639221652894307112]), '3581'), ('skip_privacy_types_IntIndexHash', 'part_1_field_3534/keys', tensor([112409236250416712]), '3534'), ('datahubLog_age_IntIndexHash', 'part_1_field_11878/keys', tensor([317212975329771157]), '11878'), ('datahubLog_gender_IntIndexHash', 'part_1_field_11880/keys', tensor([845374463820734760]), '11880'), ('mix_user_feature_education_IntIndexHash', 'part_1_field_11893/keys', tensor([788639778177576639]), '11893'), ('mix_user_feature_marriage_status_IntIndexHash', 'part_1_field_11895/keys', tensor([817113568469023338]), '11895'), ('mix_user_feature_working_status_IntIndexHash', 'part_1_field_11897/keys', tensor([598274469633128173]), '11897'), ('mix_user_feature_employment_classification_IntIndexHash', 'part_1_field_11899/keys', tensor([494111864421111778]), '11